In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from package.databases.initialize import initialize_memories
initialize_memories()

In [3]:
from package.utils.data_loder import PDFLoader
from package.interface import SourceOptions
from package.flows.offline import OfflineFlow
from package.databases.management.longterm import LongTermManagement
from package.databases.management.user import UserManagement
from package.databases.management.document import DocumentManagement
from package.databases.management.project import ProjectManagement
# from package.databases.management.jargon import JargonManagement
from package.databases.session import Session, get_session, Depends
from package.databases.models.user import User
from package.databases.models.document import Document
from package.databases.models.project import Project
# from package.databases.models.jargon import Jargon
from package.databases.models.longterm import LongTerm
from package.embedding.baai import BAAIEmbedding
from package.databases.management.term import TermManagement

embedder = BAAIEmbedding()
ltm = LongTermManagement()
dm = DocumentManagement()
# jm = JargonManagement()
um = UserManagement()
pm = ProjectManagement()
tm = TermManagement()

d:\broai-arai\backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 Loading model from: BAAI/bge-m3


Fetching 30 files: 100%|██████████| 30/30 [00:00<?, ?it/s]


In [4]:
user = User(
    username="bank",
    password="555",
    email="bank@bank.com"
)

um = UserManagement()
user = um.create_user(user, session=Depends(get_session))

In [5]:
user.id

'f7bfaa2c-9e01-4cad-8dc4-7c740caf4a9f'

In [8]:
import boto3
import json
import uuid
import os
import time
from dotenv import load_dotenv
load_dotenv()
STATE_MACHINE_ARN = os.getenv("STATE_MACHINE_ARN")

def run_stepfunctions(stateMachineArn, document_chunks):

    # Replace this with your actual Step Function ARN

    # Create a unique name for this execution
    execution_name = f"test-execution-{uuid.uuid4()}"

    # Input payload
    input_payload = {
        "document_chunks": document_chunks
    }

    # Create Step Functions client
    sfn = boto3.client("stepfunctions", region_name="ap-southeast-1")

    # Start execution
    response = sfn.start_execution(
        stateMachineArn=stateMachineArn,
        name=execution_name,
        input=json.dumps(input_payload)
    )
    return response

def check_stepfunctions_job(executionArn):
    sfn = boto3.client("stepfunctions", region_name="ap-southeast-1")
    while True:
        desc = sfn.describe_execution(executionArn=executionArn)
        status = desc["status"]
        print(f"Execution status: {status}")
        if status in ("SUCCEEDED", "FAILED", "TIMED_OUT", "ABORTED"):
            break
        time.sleep(2)

    if status == "SUCCEEDED":
        output = desc.get("output")
        print("Execution output:", output)
    else:
        print(f"Execution ended with status: {status}")    

In [9]:
from package.databases.utils import now_utc
def embed(longterms, embedder, session: Session = Depends(get_session)):
    updated_at = now_utc()
    raws = [longterm.raw for longterm in longterms]
    raw_vectors = embedder.run(sentences=raws)
    enrichs = [longterm.enrich for longterm in longterms]
    enrich_vectors = embedder.run(sentences=enrichs)
    combos = [longterm.combo for longterm in longterms]
    combo_vectors = embedder.run(sentences=combos)
    for longterm, vector in zip(longterms, raw_vectors):
        longterm.raw_embedding = vector
    for longterm, vector in zip(longterms, enrich_vectors):
        longterm.enrich_embedding = vector
    for longterm, vector in zip(longterms, combo_vectors):
        longterm.combo_embedding = vector
        longterm.updated_at = updated_at
    ltm.update_longterms(longterms=longterms, session=session)

In [10]:
from pathlib import Path
from tqdm import tqdm

source_dir = Path("./sources")
pdf_files = source_dir.glob("*.pdf")
olf = OfflineFlow()
errors = []

# target = 'Assisting in Writing Wikipedia-like Articles From Scratch with Large Language Models.pdf'
# target = 'AcuRank Uncertainty-Aware Adaptive Computation for Listwise Reranking.pdf'

for pdf_path in tqdm(pdf_files):
    source = pdf_path.name
    # if source == target:
    #     continue  # skip until we reach the target file

    source_type = pdf_path.suffix.lstrip(".")
    source_ops = SourceOptions(
        path=str(pdf_path),
        type=source_type
    )
    document = Document(source=source, type=source_type)
    document = dm.create_document(document, session=Depends(get_session))
    try:
        print(source)
        loader = PDFLoader(source=source_ops)
        contexts = loader.run()


        longterms = olf.run(document_id=document.id, contexts=contexts)
        ltm.create_raws(longterms, session=Depends(get_session))

        longterms = ltm.read_longterms_by_document(document_id=document.id, session=Depends(get_session))
        document_chunks = [{"document_id": document.id, "longterm_id": longterm.id} for longterm in longterms]

        response = run_stepfunctions(stateMachineArn=STATE_MACHINE_ARN, document_chunks=document_chunks)
        executionArn = response["executionArn"]
        check_stepfunctions_job(executionArn)

        longterms = ltm.read_longterms_by_document(document_id=document.id, session=Depends(get_session))
        embed(longterms, embedder=embedder, session=Depends(get_session))

    except Exception as e:
        errors.append((document.id, source, str(e)))

    # break  # ✅ stop the loop after processing the matched file


0it [00:00, ?it/s]

AcuRank Uncertainty-Aware Adaptive Computation for Listwise Reranking.pdf


d:\broai-arai\backend\package\utils\data_loder.py:22: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(text)
d:\broai-arai\backend\package\utils\data_loder.py:23: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)
d:\broai-arai\backend\package\utils\data_loder.py:24: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)
d:\broai-arai\backend\package\utils\data_loder.py:30: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts, max_tokens=max_tokens, overlap=overlap)


Markdown headings: max(2)
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: SUCCEEDED
Execution output: {"status":"success","results":[{"Payload":{"processed":"520ee2e0-935a-4e34-bbf4-22413a6c0713","document_id":"af617836-a979-448c-bfcf-7d6385fd12a1"}},{"Payload":{"processed":"767c1125-8733-41e6-b39e-b47ad25bd317","document_id":"af617836-a979-448c-bfcf-7d6385fd12a1"}},{"Payload":{"processed":"ad9dfe6d-b76d-48a1-a7ee-35db37935df6","document_id":"af617836-a9

pre tokenize: 100%|██████████| 2/2 [00:00<00:00, 20.17it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 2/2 [00:02<00:00,  1.06s/it]
1it [01:32, 92.66s/it]

...

17it [17:19, 54.82s/it]

TracLLM A Generic Framework for Attributing Long Context LLMs.pdf


d:\broai-arai\backend\package\utils\data_loder.py:22: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(text)
d:\broai-arai\backend\package\utils\data_loder.py:23: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)
d:\broai-arai\backend\package\utils\data_loder.py:24: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)
d:\broai-arai\backend\package\utils\data_loder.py:30: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts, max_tokens=max_tokens, overlap=overlap)


Markdown headings: max(2)
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: RUNNING
Execution status: SUCCEEDED
Execution output: {"status":"success","results":[{"Payload":{"processed":"b2330f8d-4abe-4fd7-99da-0a79d2a5c1aa","document_id":"ed281e35-924a-45e9-9fa8-d4238213437d"}},{"Payload":{"processed":"fa13257b-f14b-44a5-a65d-dbb3474ba725","document_id":"ed281e35-924a-45e9-9fa8-d4238213437d"}},{"Payload":{"processed":"a061661f-2365-4936-9388-6171775711bb","document_id":"ed281e35-924a-45e9-9fa8-d4238213437d"}},{"Payload":{"processed":"f33abbc1-02df-4990-9af4-6a5f9543f632","document_id":"ed281e35-924a-45e9-9fa8-d4238213437d"}},{"Payload":{"processed":"bc97e5d4-645f-4bf7-b3c6-570b3d7a6404","document_id":"ed281e35-924a-45e9-9fa8-d4238213437d"}},{"Payload":{"processed":"38757ba9-aeab-4d41-9137-b118d64a13a9","document_

18it [18:20, 61.13s/it]


In [12]:
errors

[]